In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
from nltk.tokenize import sent_tokenize
import nltk

# Φόρτωση μοντέλου και tokenizer
model_name = "tuner007/pegasus_paraphrase"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Συνάρτηση paraphrasing ανά πρόταση με sampling
def paraphrase_sentence_sampling(sentence, max_length=256):
    batch = tokenizer.prepare_seq2seq_batch([sentence], truncation=True, padding='longest', return_tensors="pt")
    translated = model.generate(
        **batch,
        do_sample=True,
        top_k=30,
        top_p=0.90,
        temperature=0.9,
        max_length=max_length,
        num_return_sequences=1
    )
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Paraphrasing όλων των προτάσεων ενός κειμένου (πρόταση προς πρόταση για να μη χαθεί καμία)
def paraphrase_full_text_sampling(text):
    sentences = sent_tokenize(text)
    paraphrased_sentences = []
    for i, sentence in enumerate(sentences):
        try:
            paraphrased = paraphrase_sentence_sampling(sentence)
            paraphrased_sentences.append(paraphrased)
        except Exception as e:
            print(f"Sentence {i+1} failed: {e}")
            paraphrased_sentences.append(sentence)  # fallback
    return " ".join(paraphrased_sentences)

text1 = """Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to  the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from  the professor, to show me, this, a couple of days ago.  I am very appreciated  the full support of the professor, for our Springer proceedings publication."""

text2 = """During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think. Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so. Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets."""

for i, text in enumerate([text1, text2], 1):
    print(f"\nORIGINAL TEXT {i}:\n{text}\n")
    paraphrased_text = paraphrase_full_text_sampling(text)
    print(f"PARAPHRASED TEXT {i}:\n{paraphrased_text}")
    print("="*100)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



ORIGINAL TEXT 1:
Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to  the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from  the professor, to show me, this, a couple of days ago.  I am very appreciated  the full support of the professor, for our Springer proceedings publication.

PARAPHRASED TEXT 1:
The dragon boat festival is celebrated in our Chinese culture and we should all be happy. Hope you enjoy it as much as I did. Thank you for your message, which will be shown to the doctor as his next contract checking. I saw the approved message after getting this message. I received the message from the professor a couple of days ago. The professor supported the Springer proceedings publication.

ORIGINAL TEXT 2:
During our final discuss, I told him 